In [ ]:
pip install catboost

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor

In [3]:
import pandas as pd
import numpy as np

In [29]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/test.csv')

#show dataset

In [5]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
print(train_data.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [ ]:
train_data['Street'].value_counts()

Pave    1454
Grvl       6
Name: Street, dtype: int64

In [ ]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


#подготовка датафреймов

In [5]:
def deleteId(ds):
    return ds.drop(columns='Id')

def deleteNaN(train_ds, test_ds, critval):
    fullsize = train_ds.shape[0]
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    for feature in new_train.columns:
        nulls = new_train[feature].isnull().sum()
        percent = nulls / fullsize
        # если доля пустых значений превышает critval - столбец не информативен,
        # можно его выбросить
        if (percent > critval):
            new_train = new_train.drop(columns=feature)
            new_test = new_test.drop(columns=feature)
    return new_train, new_test

def convertToNumeric(train_ds, test_ds):
    new_train = train_ds.copy()
    new_test = test_ds.copy()
    LE = LabelEncoder()
    for feature in new_train.columns[:-1]:
        if (new_train[feature].dtype == 'object'):
            new_train[feature] = LE.fit_transform(new_train[feature])
            new_test[feature] = LE.fit_transform(new_test[feature])
    return new_train, new_test

train = train_data.drop_duplicates()
train = train.drop(columns='Id')
test = test_data.drop(columns='Id')

train, test = deleteNaN(train, test, critval=0.7)
train, test = convertToNumeric(train, test)

train_y = train['SalePrice'].values
train_X = train.drop(columns='SalePrice').values
val_test = test.values

train_x, test_x, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=98987)

In [27]:
train_data['Id'].isnull().sum()

0

#Заполнение пропусков

Хорошо, но train остались пропущеные значения. Как их заполнить? 
Есть несколько тсратегий для заполнения пропущенных значений.

Заполнять пропущенные значения тем, что понравится на угад - не лучшая стратегия, часто применяют статистику для вычисления более-менее подходящего значения.

1. Самый простой и интуитивный способ - заполнить средним арифметическим значением колонки. Это не изменит среднее значение по всем значениям колонки.
2. Но иногда попадаются выбросы, поэтому, чтобы сделать вычисление среднего более робастным, можно брать усеченное среднее - сортируем значения в колонке, отбрасываем n штук слева и n штук справа и считаем среднее арифметическое.
3. Медианное среднее - медиана это точка ровно по середине выборки - сортируем значения колонки и тыкаем в середку. Такое среднее более устойчиво к выбросам, чем среднее арифметическое.
4. Наиболее часто встрчаемое - еще одна стратегия для заполнения - если чаще всего встречается - почему бы и нет.

### 1. Когда находим пропуск, можно искать наиболее похожую строку в датасете и заполнить этим значением. Близость можно определить расстоянием в пространстве значений, например евклидово расстояние или косинусное расстояние.

In [43]:
#найдем все колонки с пропущенными данными
def find_nan_columns(dataframe):  # возращаем список в виде [название колонки, процент пропущенных значений]
    fullsize = dataframe.shape[0]
    nan_columns = []
    rows_with_nan = []
    new_train = dataframe.copy()
    for column in dataframe.columns:
        nulls = new_train[column].isnull().sum()
        if nulls != 0:
            nan_columns.append([column, nulls / fullsize * 100])
    return nan_columns

def fill_data_gaps(dataframe, nan_columns):
    _df = dataframe.copy()
    for i in range(len(nan_columns)):
        # начинаем обучать с колонки с наименьшим кол-вом пропущенных значений

        # выбрасываем колонки с пропущенными значениями: 
        # при этом уже обученные колонки не выбрасываем
        if i+1 != len(nan_columns[i:]):
            for column in nan_columns[i+1:]:
                _df = _df.drop(columns=column[0])

        # берем только строки без пропусков
        not_nan_df = _df[_df[nan_columns[i][0]].notnull()]
        trainY = not_nan_df[nan_columns[i][0]].values
        trainX = not_nan_df.drop(columns = nan_columns[i][0]).values
        # обучаем регрессор
        model = LogisticRegression(max_iter=1000, random_state=42)
        model.fit(trainX, trainY)
        # теперь заполним значения
        rows = _df[nan_columns[i][0]].isnull()
        for i in range(len(rows)):
            if rows[i]:
                _df.loc[i, nan_columns[i][0]] = model.predict(_df.loc[i].drop(columns=nan_columns[i][0]).values)
    return _df

       

train = train_data.drop_duplicates()
train = train.drop(columns='Id')
test = test_data.drop(columns='Id')

train, test = deleteNaN(train, test, critval=0.7)
train, test = convertToNumeric(train, test)

nan_columns = find_nan_columns(train)
nan_columns.sort(key = lambda x: x[1])

fill_data_gaps(train, nan_columns).head()

In [60]:
x = [0, 1, 2]
print(x[0:])
print(x[1:])
print(x[2:])

[0, 1, 2]
[1, 2]
[2]


###2.Итеративное обучение моделей на непропущенных данных для предсказания пропущенных данных.

Например, если у нас 10 колонок, в первой колонке пропущенно x% данных, во второй - у%, в четвертой - с%. Тогда для заполнения пропущенных данных для четвертой колонки возбмем данные из колонок: 3, 5-10, обучим модельку и заполним предсказаниями. для второй - 3-10 колонки и тоже обучим, для первой - 2-10.

In [59]:
#найдем все колонки с пропущенными данными
def find_nan_columns(dataframe):  # возращаем список в виде [название колонки, процент пропущенных значений]
    fullsize = dataframe.shape[0]
    nan_columns = []
    rows_with_nan = []
    new_train = dataframe.copy()
    for column in dataframe.columns:
        nulls = new_train[column].isnull().sum()
        if nulls != 0:
            nan_columns.append([column, nulls / fullsize * 100])
    return nan_columns

def fill_data_gaps(dataframe, nan_columns):
    _df = dataframe.copy()
    for i in range(len(nan_columns)):
        # начинаем обучать с колонки с наименьшим кол-вом пропущенных значений

        # выбрасываем колонки с пропущенными значениями: 
        # при этом уже обученные колонки не выбрасываем
        if i != len(nan_columns[i:]):
            for column in nan_columns[i+1:]:
                _df = _df.drop(columns=column[0])

        # берем только строки без пропусков
        not_nan_df = _df[_df[nan_columns[i][0]].notnull()]
        trainY = not_nan_df[nan_columns[i][0]].values
        trainX = not_nan_df.drop(columns = nan_columns[i][0]).values
        # обучаем регрессор
        model = LogisticRegression(max_iter=1000, random_state=42)
        model.fit(trainX, trainY)
        # теперь заполним значения
        rows = _df[nan_columns[i][0]].isnull()
        for i in range(len(rows)):
            if rows[i]:
                _df.loc[i, nan_columns[i][0]] = model.predict(_df.loc[i].drop(columns=nan_columns[i][0]).values)
    return _df

       

train = train_data.drop_duplicates()
train = train.drop(columns='Id')
test = test_data.drop(columns='Id')

train, test = deleteNaN(train, test, critval=0.7)
train, test = convertToNumeric(train, test)

nan_columns = find_nan_columns(train)
nan_columns.sort(key = lambda x: x[1])

fill_data_gaps(train, nan_columns).head()

###Стандартный подход

In [ ]:
strategies = ['mean', 'median', 'most_frequent']
imputer = SimpleImputer(strategy=strategies[2])
trainX = imputer.fit_transform(train_x)
testX = imputer.fit_transform(test_x)

In [ ]:
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

In [ ]:
trainX.shape

(1314, 75)

#RandomForest

In [ ]:
parameters = {
    'criterion':(['squared_error']), 
    'max_depth':  range(1000, 3001, 500),
    'n_estimators': range(10, 51, 10),
    'min_samples_leaf': range(1, 9, 2),
    'min_samples_split': range(2, 11, 2)}

model = RandomForestRegressor()
rf_gs_model = GridSearchCV(model, parameters)
rf_gs_model.fit(trainX[:600], train_y[:600])
print(f"Best parameters: {rf_gs_model.best_params_}")

In [ ]:
def check_error(preds, gt):
    print('RMSE Error:', mean_squared_error(np.log(preds), np.log(gt), squared=False))

In [ ]:
last_rf_best_params = {
    'criterion': 'squared_error', 
    'max_depth': 1500,
    'n_estimators': 40,
    'min_samples_leaf': 1,
    'min_samples_split': 2
}

random_forest = RandomForestRegressor(**last_rf_best_params)  # (**rf_gs_model.best_params_)
random_forest.fit(trainX, train_y)
check_error(random_forest.predict(testX), test_y)

RMSE Error: 0.15719618143042727


#XGBoost

Бустинг - метод ансамблирования, в котором обучается много копий более слабой модели ("weak learner"), т.е. такой модели, которая не может достичь высокой точности на обучающей выборке, переобучившись на ней. Например, решающее дерево небольшой глубины. На каждом шаге новая модель концентрируется на исправлении ошибок предыдущей модели. Итоговое предсказание получается как взвешенное голосование всех моделей. Таким образом, главное отличие Бустинга от бэггинга - обучение моделей происходит зависимо от предыдущих моделей и последовательно, когда в бэггинге обучение происходит независимо и параллельно.

XGBoost - вычислительно эффективная реализация градиентного бустинга над решающими деревьями.

Суть градинетного бустинга в следующем: каждый последующий "слабый" алгоритм должен корректировать предсказания предыдущего. При этом входными параметрами следующего алгоритма является минус градиент функции потерь по предсказаниям предыдущего алгоритма. При этом от функции потерь не трубуется дифференцируемость по параметрам в отличе от градиентного спуска, но требуется диференцируемость по предсказаниям.

Пусть:
- $\{x_i, y_i\}_{i=1}^N$ - обучающая выборка
- K - кол-во деревьев в ансамбле
- $f_k:x \rightarrow y$ - k-e дерево ансамбля как функция
- $F: x → y$ - весь ансамбль как функция
- $loss: y_{true} \times y_{pred} → R$ - какая-то функция потерь
- $T_k$ - кол-во листьев в к-ом дереве
- $w_k$ - вектор, составленный из выходных значений на всех листьях к-го дерева

в XGBoost ответы суммируются по всем деревьям ансамбля: $$F(x) = Σ_{k=1}^Kf_x(x)$$

Суммарная функция потерь выглядит так: $$Loss_{total} = Σ_{i=1}^Nloss(y_i, F(x_i)) + γΣ_{k=1}^KT_k + \frac{1}{2}λΣ_{k=1}^K||w_k||^2,$$
где γ, λ - гиперпараметры.

Первое слагаемое - суммируем все потери по всем деревьям,

Второе слагаемое - штрафует деревья за слишком большое кол-во листьев,

Третье слагаемое - обеспечивает механизм, чтобы каждое дерево вносило минимальный вклад в результат.

Для минимизации $Loss_{total}$ используются первая и вторая производные по предсказаниями.

**Построение деревьев**

Введем целевую функцию (objective function): $$obj = Σ_{i=1}^Nloss(y_i, y_i^{'(t)} + Σ_{i=1}^tw(f_i)) $$

Каждое следующее дерево должно улучшать предсказание предыдущего:
$$y_i^{'(t+1)} = y_i^{'(t)} + f_{t+1}(x_i)$$

Вопрос: как строить дерево на каждой следующей итерации? Интуитивный ответ - строить так, чтобы оптимизировать целевую функцию:

$obj^{(t)} = Σ_{i=1}^Nloss(y_i, y_i^{'(t)}) + Σ_{i=1}^tw(f_i) = Σ_{i=1}^Nloss(y_i, y_i^{'(t-1)} + f_t(x_i)) + Σ_{i=1}^tw(f_i) + constant$

если, например, использовать MSE:

$obj^{(t)} = Σ_{i=1}^N(y_i - ( y_i^{'(t-1)} + f_t(x_i)))^2 + Σ_{i=1}^tw(f_i) = Σ_{i=1}^N[(y_i - y_i^{'(t-1)})^2 + 2(y_i - y_i^{'(t-1)})f_t(x_i) + f_t(x_i)^2] + Σ_{i=1}^tw(f_i) + constant$

в случае MSE получается полегче т.к. сама функция квадратичная, а основание 
линейное. В случае других функций потерь пользуемся разложением в ряд Тейлора.

$obj^{(t)} = Σ_{i=1}^N[loss(y_i, y_i^{'(t-1)}) + g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i)] + w(f_t) + const$, где:

$$g_i = \frac{dloss(y_i, y_i^{'(t-1)})}{dy_i^{'(t-1)}}$$
$$h_i = \frac{d^2loss(y_i, y_i^{'(t-1)})}{d(y_i^{'(t-1)})^2}$$

После приведения всех слогаемых, получаем оптимизационную задачу для построения нового дерева:

$$Σ_{i=1}^N[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)] + w(f_t)$$

Сложность будем определять следующим образом:

$$w(f) = γT + \frac{1}{2}λΣ_{j=1}^Tw_j^2$$, где w - вектор значений листьев, T - кол-во листьев

Тогда с учетом вышеуказанной регуляризации перепишем нашу целевую функцию:

$$obj^{(t)} = Σ_{i=1}^N[g_if_t(x_i)+\frac{1}{2}h_if_t^2(x_i)] + γT + \frac{1}{2}λΣ_{j=1}^Tw_j^2 = Σ_{j=1}^T[(Σ_{i из I_j}g_i)w_i + \frac{1}{2}(Σ_{i из I_j}h_i + λ)w_j^2] + γT$$, где $I_j = \{i|q(x_i)=j\}$ - множество элементов выборки, относящихся к j-ому листу.


положим $G_i =Σ_{i из I_j}g_i $ и $H_j=Σ_{i из I_j}h_i$, тогда

$$obj^{(t)} = Σ_{j=1}^T[G_jw_j + 1/2(H_j + λ)w_j^2] + γT$$ - получили функцию, которая квадратично зависит от $w_j$ - поэтому можем легко найти наилучшую оценку:

$$w_j^* = -\frac{G_j}{H_j + λ}$$
$$obj^* = -\frac{1}{2}Σ_{j=1}^T\frac{G_i^2}{H_j+λ}+γT$$ - измеряет, насколько построенное дерево "хорошее"

$$IG = \frac{1}{2}[\frac{G_L^2}{H_L+λ} + \frac{G_R^2}{H_R+λ} - \frac{(G_L + G_R)^2}{H_L+H_R+λ}] - γ$$

- первое слагаемое показывает оценку левого листа
- второе слагаемое оценку правого листа
- третье слагаемое оценка исходного листа
- четвертое слагаемое - регуляризация добавленного листа

Заметим, если прирост меньше чем регуляризация - мы не будем делить лист на две части.

Одним из важных настраиваемых параметров является **subsample** - которые указывает размер подвыборки но без возврата, и learning rate конечно.

In [ ]:
parameters = {
    "learning_rate": np.linspace(0.1, 0.5, 10),
    "max_depth": range(3, 10, 1),
    "min_child_weight": range(5, 16, 2),
    "n_estimators": range(30, 91, 10),
    "subsample": (0.63, 0.7, 0.75, 0.81)
}

model = XGBRegressor()
xgb_gs_model = GridSearchCV(model, parameters)
xgb_gs_model.fit(trainX[:600], train_y[:600])
print(f"Best parameters: {xgb_gs_model.best_params_}")

In [ ]:
last_xgb_best_params = {
    "learning_rate": 0.15,
    "max_depth": 3,
    "min_child_weight": 6,
    "n_estimators": 50, 
    "subsample": 0.75,
    "booster": 'gbtree',
    # "reg_lambda": 1
    "gamma": 0
}

xgb_reg = XGBRegressor(**last_xgb_best_params)  #(**xgb_gs_model.best_params_)
xgb_reg.fit(trainX, train_y)
check_error(xgb_reg.predict(testX), test_y)

[07:05:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE Error: 0.14302138559911587


In [ ]:
xgb_reg.predict(val_test)

array([425859.2 , 375549.97, 427963.6 , ..., 421054.1 , 303494.72,
       407534.2 ], dtype=float32)

#LightGBM

**LightGBM** - фокусируется на примерах бустинга с большими градиентами. Реализация LigthGBM вводит две ключевые идеи: GOSS и EFB.

1. GOSS - градиентная односторонняя выборка - фокусируется на тех учебных примерах, которые приводят к большему градиенту, ускоряя обучения и уменьшая вычислительную сложность алгоритма. Т.е. мы просто используем элементы выборки с наибольшим градиентом, ведь они имеют наибольшее значение для IG.
2. EFB - объединение взаимоисключающих признаков - подход объединения разрезженных взаимоисключающих признаков.

LightGBM создает деревья решений, которые растут по листам, что означает, что при заданом условии разделяется только один лист.

In [ ]:
params = {
    "learning_rate": [0.2, 0.1, 0.01],
    "num_leaves": range(3, 10, 2),
    "max_depth": range(3, 10 ,2),
    "min_data_in_leaf": range(5, 100, 5),
    "n_estimators": range(30, 91, 10)
}

model = LGBMRegressor()
gs_model = GridSearchCV(model, params)
gs_model.fit(trainX[:300], train_y[:300])
print(f'Best params: {gs_model.best_params_}')

Best params: {'learning_rate': 0.1, 'max_depth': 3, 'min_data_in_leaf': 25, 'n_estimators': 60, 'num_leaves': 3}


In [ ]:
last_lightgbm_best_params = {
    "learning_rate": 0.15,
    "num_leaves": 6,
    "max_depth": 3,
    "min_data_in_leaf": 25,
    "n_estimators": 70
}

model = LGBMRegressor(**last_lightgbm_best_params)  # (**gs_model.best_params_, eval_metric='rmse')
model.fit(trainX, train_y)
check_error(model.predict(testX), test_y)

RMSE Error: 0.1469000875271349


#CatBoost

CatBoost - реализация градиентного бустинга над решающими деревьями.

Основные особенности:
- Использование решающих таблиц
- Упорядоченное target-кодирование на категориальных признаках высокой размерности
- Бустинг с упорядочиванием обучающих примеров

*target-кодирование* - это когда мы вместо категориального признака ставим некоторое статистически оцененное значение целевого признака, например, вместо марки машины - среднюю цену всех авто.

Обозначим две проблемы:
1. Переобучение (смещенность) в градиентах: т.к. каждый следующий алгоритм обучается на градиенте, в котором уже спрятано целевое значение. - Хочется как-то делать обучение на разных данных, но как? - Для получения несмещенных оценок на объекте $x_i$ храним и достраиваем ансамбль только на выборке без этого объекта - похоже на out-of-bag. Но таких выборок может получиться очень много и очень похожих.
2. Надо обрабатывать категориальные признаки с большим числом редких значений.

**Решающая таблица** - частный случай забывчивого дерева - все решающие правила одного уровня (на одинаковом расстоянии от корня) проверяют один и тот же признак. В CatBoost - на каждом уровне решающего дерева используется не только общий признак, но и общий порог разделения.

**Упорядоченный бустинг** - способ решения переобучения на градиентном бустинге.
В CatBoost на каждом шаге бустинга обучается одна решающая таблица(дерево), глубина дерева является гиперпараметром. Если глубина N - N признаков и $2^N$ листьев.

Идея:
- градиент $i$-го объекта вычисляем по модели, которая на этом объекте не обучалась.accuracy_score
- строим подвыборку удваивающейся длины: $j=1$ длины 1, $j=2$ длины 2, $j=3$ длины 4, $j=4$ длины 8, ..., $j=n$ длины $2^{n-1}$
- строим много таких случайных выборок - делаем перестановки - из каждой перестаовки вытаскиваем подвыборку для обучения.

Пусть сделаем s случайных перестоновок, r - номер перестановки.

тогда для какой-то одной случайной выбраной перестановки рассчитывает градиента: $g_{ti} = Loss^{'}(a_{t-1}^{rj}(x_i), y_i)$ - градиент в точке $(x_i, y_i)$, $j = log_2(i-1)$, t - внутренняя итерация, строим по подвыборке не содержащей этот $x_i$

**Алгоритм:**
 1. генерируем выборки для всех перестоновок.
 2. для всех t = 1, ..., T:
  - выбраем выборку построенную с перестоновкой $σ_r$
  - вычисляем несмещенный вектор градиента: все элементы одной подвыборки обучаются на алгоритмах не содержащих элементы данной подвыборки:
  $$b_t = argmin_b Σ_{i=1}^l(b(x_i) + g_{ti})^2$$ - так мы находим новое дерево - t-ый базовый алгоритм
  теперь для всех деревьев $b_t^{ri}$ мы устанавливаем одинаковую структуру дерева - вершины и условия ветвления. Но в каждом дереве вычисляем значения в листьях индивидуально - значения элементов в листбях у каждого дерева свои.
  Вычисляем значения в листьях для базового алгоритма на отдельной перестановке выборки и вычисляем как в обычном бустинге вычисляем веса алгоритмов и обновляем алгоритм.

- категориальные признаки тоже оцениваются как несмещенная оценка - для $x_i$ элмента выборки категориальный признак оценивается по статистике из элементов $x_j| j < i$

Алгоритм:
- На вход подается обучающая выборка, количество шагов, функция потерь, learning rate и количество перестоновок минус единицa s
- Сначала создается s+1 перестановка - упорядочивается обучающая выборка.
Позицию i-го элемента выборки в перестановке $σ_r$ обозначим как $σ_r(i)$ Эти перестановки применяются как для упорядоченного бустинга, так и для упорядоченного target-кодирования.

Catboost на каждом шаге бустинга обучает одно дерево, но при этом иммитирует обучение сразу многих "виртуальных" моделей градиентного бустинга. Поэтому результатом на каждом шаге является сразу несоклько предсказаний для каждого примера, результаты хранятся в массиве М. При этом запись $M_{r, j}(i)$ - предсказание на $i$-ом перемере, $r$ - номер перестановки, $j$ - соответствует порядку к упорядоченном бустинге.

- при $N$ обчающих элементов, Catboost строит только $log_2n$ моделей.
- Каждое новое дерево строится с учетом предыдущих предсказаний

In [ ]:
train = train_data.drop_duplicates()
train = train.drop(columns='Id')
test = test_data.drop(columns='Id')

train, test = deleteNaN(train, test, critval=0.7)

categorical = []
features = train.columns
for i in range(len(features)):
    if (train[features[i]].dtype == 'object'):
        categorical.append(i) 

train_y = train['SalePrice'].values
train_X = train.drop(columns='SalePrice').values
val_test = test.values

train_x, test_x, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=98987)

imp = SimpleImputer(strategy='most_frequent')
trainX = imp.fit_transform(train_x)
testX = imp.fit_transform(test_x)

In [ ]:
# features = {
#     "loss_function":'RMSE', 
#     "cat_features":categorical,
#     "learning_rate": [0.3, 0.2, 0.1],
#     "depth": range(2, 11, 2),
#     "bootstrap_type":'Bernoulli',
#     "grow_policy":'SymmetricTree',
#     "l2_leaf_reg": [0.2, 0.4, 0.6, 0.7]
# }

# model = CatBoostRegressor()
# gs_model = GridSearchCV(model, params)
# gs_model.fit(trainX[:300], train_y[:300])
# print(f'Best params: {gs_model.best_params_}')

In [ ]:
cat_model = CatBoostRegressor(loss_function='RMSE', 
                              bagging_temperature=150,
                              random_strength=7,
                              cat_features=categorical,
                              learning_rate=0.075,
                              depth=3,
                              iterations=1000,
                            #   bootstrap_type='Bernoulli',
                              grow_policy='SymmetricTree',
                              l2_leaf_reg=0.55,
                            #   plot=True
                              )
cat_model.fit(trainX, train_y)
check_error(cat_model.predict(testX), test_y)

#Стэккинг

Что такое стэкинг? Стэкинг - способ ансамблирования моделей, в которой в роли агрегатной функции выступает другой, так называемый, **мета-алгоритм**.

In [ ]:
class StackRegressor:
    def __init__(self, params_randomforest=None, params_lightgbm = None, 
                 params_xgboost=None, params_catboost=None) -> None:

        self.model0, self.model1, self.model2, self.model3 = None, None, None, None
        self.meta_model = None
        self.lr, self.iter = None, None
        self.params_randomforest = 0
        if params_randomforest is not None:
            self.params_randomforest = params_randomforest
        self.params_lightgbm = 0
        if params_lightgbm is not None:
            self.params_lightgbm = params_lightgbm
        self.params_xgboost = 0
        if params_xgboost is not None:
            self.params_xgboost = params_xgboost
        self.params_catboost = 0
        if params_catboost is not None:
            self.params_catboost = params_catboost

    def fit(self, trainX, trainY, div) -> None:
        trainX0 = trainX[:int(div*len(trainY))]
        trainY0 = trainY[:int(div*len(trainY))]
        trainX_meta = trainX[int(div*len(trainY)):]
        trainY_meta = trainY[int(div*len(trainY)):]

        self.model0 = RandomForestRegressor(**self.params_randomforest)
        self.model0.fit(trainX0, trainY0)

        self.model1 = LGBMRegressor(**self.params_lightgbm, eval_metric='rmse')
        self.model1.fit(trainX0, trainY0)

        self.model2 = XGBRegressor(**self.params_xgboost)
        self.model2.fit(trainX0, trainY0)  

        # self.model3 = CatBoostRegressor(loss_function='RMSE', 
        #                   cat_features=categorical,
        #                   learning_rate=0.05,
        #                   depth=3,
        #                   bootstrap_type='Bernoulli',
        #                   grow_policy='SymmetricTree',
        #                   l2_leaf_reg=0.25)
        # self.model3.fit(trainX0, trainY0)

    def predict(self) -> None:
        pass

In [ ]:
def data_for_gboosting():
    train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/train.csv')
    test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/test.csv')

    train = train_data.drop_duplicates()
    train = train.drop(columns='Id')
    test = test_data.drop(columns='Id')

    train, test = deleteNaN(train, test, critval=0.7)
    train, test = convertToNumeric(train, test)

    train_y = train['SalePrice'].values
    train_X = train.drop(columns='SalePrice').values
    val_test = test.values

    train_x, test_x, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=98987)

    strategies = ['mean', 'median', 'most_frequent']
    imputer = SimpleImputer(strategy=strategies[2])
    trainX = imputer.fit_transform(train_x)
    testX = imputer.fit_transform(test_x)

    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.transform(testX)

    # return (trainX[int(0.1)*len(train_y):], 
    #         trainX[:int(0.1)*len(train_y)], 
    #         train_y[int(0.1)*len(train_y):], 
    #         train_y[:int(0.1)*len(train_y)], 
    #         testX[int(0.1)*len(test_y):],
    #         testX[:int(0.1)*len(test_y)],
    #         test_y[int(0.1)*len(test_y):],
    #         test_y[:int(0.1)*len(test_y)])
    return trainX, train_y, test_x, test_y


def data_for_catboosting():
    train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/train.csv')
    test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML 4 course/test.csv')

    train = train_data.drop_duplicates()
    train = train.drop(columns='Id')
    test = test_data.drop(columns='Id')

    train, test = deleteNaN(train, test, critval=0.7)

    categorical = []
    features = train.columns
    for i in range(len(features)):
        if (train[features[i]].dtype == 'object'):
            categorical.append(i) 

    train_y = train['SalePrice'].values
    train_X = train.drop(columns='SalePrice').values
    val_test = test.values

    train_x, test_x, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=98987)

    imp = SimpleImputer(strategy='most_frequent')
    trainX = imp.fit_transform(train_x)
    testX = imp.fit_transform(test_x)

    return trainX, train_y, testX, test_y, categorical

In [ ]:
# trainx_0, test_0, train_y0, test_y0, trainx_1, testx_1, train_y1, test_y1= data_for_gboosting()
# cat_trainX0, cat_trainY0, categorical = data_for_catboosting()
trainX0, trainy0, trainX1, trainy1 = data_for_gboosting()
cat_trainX0, cat_trainy0, cat_trainX1, cat_trainy1, categorical = data_for_catboosting()

In [ ]:
rf_model = RandomForestRegressor(**last_rf_best_params)
rf_model.fit(trainX0, trainy0)

lgbm_model = LGBMRegressor(**last_lightgbm_best_params, eval_metric='rmse')
lgbm_model.fit(trainX0, trainy0)

xgb_model = XGBRegressor(**last_xgb_best_params)
xgb_model.fit(trainX0, trainy0) 

# cat_model = CatBoostRegressor(loss_function='RMSE', 
#                               bagging_temperature=150,
#                               random_strength=7,
#                               cat_features=categorical,
#                               learning_rate=0.1,
#                               depth=3,
#                               iterations=2500,
#                             #   bootstrap_type='Bernoulli',
#                               grow_policy='SymmetricTree',
#                               l2_leaf_reg=0.55)
# cat_model.fit(cat_trainX0, cat_trainy1)

[07:16:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(learning_rate=0.15, min_child_weight=6, n_estimators=50,
             subsample=0.75)

In [ ]:
predictions = []
for i in range(len(trainy1)):
    predictions.append([rf_model.predict(trainX1[i]), 
                       lgbm_model.predict(trainX1[i]),
                       xgb_model.predict(trainX1[i])])  #, cat_model.predict(cat_trainX1[i])])

#Kaggle

In [ ]:
strategies = ['mean', 'median', 'most_frequent']
imputer = SimpleImputer(strategy=strategies[2])
val_test_x = imputer.fit_transform(val_test)

scaler = StandardScaler()
val_test_x = scaler.fit_transform(val_test_x)

In [ ]:
submission_path = '/content/drive/MyDrive/Colab Notebooks/ML 4 course/sample_submission.csv'
submission = pd.read_csv(submission_path)

In [ ]:
random_forest.predict(val_test_x)

In [ ]:
xgboost_predictions = xgb_reg.predict(val_test_x)

In [ ]:
submission['SalePrice'] = xgboost_predictions
submission
submission.to_csv('submission_xgboost.csv', index=False)

In [ ]:
train = train_data.drop_duplicates()
train = train.drop(columns='Id')
test = test_data.drop(columns='Id')

train, test = deleteNaN(train, test, critval=0.7)

categorical = []
features = test.columns
for i in range(len(features)):
    if (test[features[i]].dtype == 'object'):
        categorical.append(i) 

val_test = test.values
imp = SimpleImputer(strategy='most_frequent')
val_test_x = imp.fit_transform(val_test)
# testX = imp.fit_transform(test_x)

In [ ]:
catboost_predictions = cat_model.predict(val_test_x)

In [ ]:
catboost_predictions[3]

195791.94291769294

In [ ]:
submission['SalePrice'] = catboost_predictions
submission

,Id,SalePrice
0,1461,118364.122412
1,1462,161312.646421
2,1463,197618.872119
3,1464,195791.942918
4,1465,193338.562441
...,...,...
1454,2915,87136.393590
1455,2916,83774.633636
1456,2917,155468.490510
1457,2918,120404.556310


In [ ]:
submission.to_csv('submission_catboost.csv', index=False)